In [49]:
import cv2
import numpy as np
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
# Callbacks 
 
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True),
    ModelCheckpoint(filepath='model.keras', save_best_only=True),
]

In [51]:
# Charger les données d'entraienement 

def load_data():
    # Initialiser les listes pour les données et les labels
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    # Dossier contenant les images d'entraînement et de test
    #for photos in ['oliwer', 'damien']:
    for photos in ['oliwer']:
        train = f'processed/{photos}/train/' # Images d'entraînement

        for file in os.listdir(train):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = os.path.join(train, file) # Lire le chemin de l'image
                image = cv2.imread(img, cv2.IMREAD_GRAYSCALE) # Transformer l'image en gris
                train_data.append(image) # Ajouter l'image à la liste des données d'entraînement
                train_labels.append(photos) # Ajouter le label correspondant

        for file in os.listdir(f'processed/{photos}/test/'):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = os.path.join(f'processed/{photos}/test/', file) # Lire le chemin de l'image de test
                image = cv2.imread(img, cv2.IMREAD_GRAYSCALE) # Transformer l'image en gris
                test_data.append(image) # Ajouter l'image à la liste des données de test
                test_labels.append(photos) # Ajouter le label correspondant

    # Données aléatoires
    random_dir = './lfw-deepfunneled/lfw-deepfunneled'  # Dossier avec des images aléatoires

    # Parcourir les sous-dossiers pour ajouter des images aléatoires
    random_images = []
    for person in os.listdir(random_dir):
        person_path = os.path.join(random_dir, person) # Chemin du sous-dossier de la personne
        if os.path.isdir(person_path):
            for filename in os.listdir(person_path):
                img_path = os.path.join(person_path, filename) # Chemin complet de l'image
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # Transformer l'image en gris
                    image = cv2.resize(image, (128, 128)) # Redimensionner l'image à 100x100 pixels
                    random_images.append(image) # Ajouter l'image à la liste des images aléatoires

    # Sélectionner 80 images pour l'entraînement et 20 pour le test
    np.random.shuffle(random_images)
    train_data.extend(random_images[:80])
    train_labels.extend(['random'] * 80)
    test_data.extend(random_images[80:100])
    test_labels.extend(['random'] * 20)

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

In [ ]:
train_data, train_labels, test_data, test_labels = load_data() # Charger les données

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)  # Encoder les labels d'entraînement
test_labels = label_encoder.transform(test_labels)        # Encoder les labels de test

train_labels = to_categorical(train_labels, num_classes=3)  # Convertir en one-hot encoding
test_labels = to_categorical(test_labels, num_classes=3)    # Convertir en one-hot encoding

train_data = train_data.reshape(-1, 128, 128, 1)  # Reshape pour le modèle CNN
test_data = test_data.reshape(-1, 128, 128, 1)    # Reshape pour le modèle CNN

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128,128,1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='sigmoid')  # 2 classes: 'oliwer', 'random'
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=50, batch_size=32, validation_data=(test_data, test_labels), callbacks=callbacks)


Epoch 1/50


c:\Users\Oliwer\Documents\facial_recognition\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step - accuracy: 0.6281 - loss: 14.6969 - val_accuracy: 0.7500 - val_loss: 1.7848 - learning_rate: 0.0010
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.8906 - loss: 0.9393 - val_accuracy: 0.9750 - val_loss: 0.0620 - learning_rate: 0.0010
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.9656 - loss: 0.0812 - val_accuracy: 0.9875 - val_loss: 0.0210 - learning_rate: 0.0010
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.9844 - loss: 0.0226 - val_accuracy: 1.0000 - val_loss: 5.4033e-04 - learning_rate: 0.0010
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 2.6678e-04 - learning_rate: 0.0010
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 1.0466e-05 - learning_rate: 0.0010
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 1.0000 - loss: 8.5034e

In [56]:
loaded_model = keras.models.load_model('modelSigmoid.keras')  # Charger le modèle sauvegardé

# Évaluation du modèle
test_loss, test_accuracy = loaded_model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

test_file = '20250731_114033_087.jpg'

test_image = cv2.imread(test_file, cv2.IMREAD_GRAYSCALE)
test_image = cv2.resize(test_image, (128, 128))
test_image = test_image.reshape(1, 128, 128, 1)  # Reshape pour le modèle CNN
predictions = loaded_model.predict(test_image)
predicted_label = label_encoder.inverse_transform([np.argmax(predictions[0])])
print(f"Predicted label: {predicted_label[0]}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 4.6640e-07 
Test Loss: 4.664024402245559e-07, Test Accuracy: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted label: oliwer
